# Burgers Equation Identification
Equation:   $u_{t} + \lambda_1 uu_{x}-\lambda_2 u_{xx} = 0$  

In [3]:
#author : $um@nth
import torch
import torch.nn as nn
import numpy as np
from torch.autograd import grad
import scipy.io
from torch.utils.data import Dataset, DataLoader

In [17]:
N_u = 2000
layers = [2, 25, 25, 25, 25, 25, 25, 25, 25, 1]
data = scipy.io.loadmat('burgers_shock.mat') 
t = data['t'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = np.real(data['usol']).T
X, T = np.meshgrid(x,t)
X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))                  
u_star = Exact.flatten()[:,None]                                                 
lb = X_star.min(0)                                                              
ub = X_star.max(0)
np.random.seed(107)
idx = np.random.choice(X_star.shape[0], N_u, replace=False)

X_train = X_star[idx,:]
u_train = torch.from_numpy(u_star[idx,:]).float()
X = torch.from_numpy(X_train[:,0:1]).requires_grad_(True).float()     
T = torch.from_numpy(X_train[:,1:2]).requires_grad_(True).float()

In [18]:
class PINN(nn.Module):

  def __init__(self, layers):
    super(PINN, self).__init__()
    self.layers = nn.ModuleList()
    for i, j in zip(layers, layers[1:]):
      linear = nn.Linear(i, j)
      nn.init.xavier_normal_(linear.weight.data, gain = 1.0)
      nn.init.zeros_(linear.bias.data)
      self.layers.append(linear)
  
  def forward(self, x):
    L = len(self.layers)
    for l, transform in enumerate(self.layers):
      if l < L-1:
        x = torch.tanh(transform(x))
      else:
        x = transform(x)
    return x   

In [34]:
class PINN_Run():

  def __init__(self, X, T, model, u_train):
    self.X = X
    self.T = T
    self.u_train = u_train
    self.model = model
    self.i = 0
    self.l1 = nn.Parameter(torch.tensor([0.], requires_grad=True))
    self.l2 = nn.Parameter(torch.tensor([0.], requires_grad=True))
    self.model.register_parameter('lambda_1', self.l1)
    self.model.register_parameter('lambda_2', self.l2)
    self.optimizer = torch.optim.LBFGS(
            self.model.parameters(), 
            lr = 1, 
            max_iter = 20000, 
            max_eval = 20000, 
            history_size = 50,
            tolerance_grad = 1e-5, 
            tolerance_change = 1.0 * np.finfo(float).eps,
            line_search_fn = "strong_wolfe"
        )
  
  def residual_loss(self):
    xf = torch.cat([self.X, self.T], axis=1)
    uf = self.model(xf)
    u_x = grad(uf.sum(), self.X, retain_graph = True, create_graph = True)[0]
    u_xx = grad(u_x.sum(), self.X, retain_graph = True, create_graph = True)[0]
    u_t = grad(uf.sum(), self.T, retain_graph = True, create_graph = True)[0]
    f = u_t + self.l1*uf*u_x - self.l2*u_xx 
    return torch.mean(torch.square(f))
  
  def closure(self):
    self.model.train()
    mse = nn.MSELoss()
    self.optimizer.zero_grad()
    yhat = self.model(torch.cat([self.X, self.T], axis=1).float())
    loss1 = mse(yhat, self.u_train)
    loss2 = self.residual_loss()
    loss = loss1 + loss2
    loss.backward()
    if self.i % 100 == 0:
      print('Epoch:', self.i, 'Loss: %.5e, Lambda_1: %.5f, Lambda_2: %.5f' % (loss.item(), self.l1, self.l2))
    self.i += 1
    return loss
  
  def train_(self):
    self.optimizer.step(self.closure)

In [35]:
pinn_model = PINN(layers)
pinn_ = PINN_Run(X, T, pinn_model, u_train)
pinn_.train_()

Epoch: 0 Loss: 4.74090e-01, Lambda_1: 0.00000, Lambda_2: 0.00000
Epoch: 100 Loss: 3.15923e-02, Lambda_1: 0.09032, Lambda_2: 0.00187
Epoch: 200 Loss: 2.44118e-02, Lambda_1: 0.15859, Lambda_2: 0.00090
Epoch: 300 Loss: 2.44214e-02, Lambda_1: 0.40701, Lambda_2: 0.00487
Epoch: 400 Loss: 1.79056e-02, Lambda_1: 0.45164, Lambda_2: 0.00372
Epoch: 500 Loss: 1.28324e-02, Lambda_1: 0.65857, Lambda_2: 0.00707
Epoch: 600 Loss: 9.06787e-03, Lambda_1: 0.68401, Lambda_2: 0.00594
Epoch: 700 Loss: 7.32466e-03, Lambda_1: 0.69197, Lambda_2: 0.00470
Epoch: 800 Loss: 5.86729e-03, Lambda_1: 0.77323, Lambda_2: 0.00480
Epoch: 900 Loss: 4.67202e-03, Lambda_1: 0.83809, Lambda_2: 0.00462
Epoch: 1000 Loss: 4.08641e-03, Lambda_1: 0.84423, Lambda_2: 0.00476
Epoch: 1100 Loss: 3.61925e-03, Lambda_1: 0.84939, Lambda_2: 0.00465
Epoch: 1200 Loss: 3.11064e-03, Lambda_1: 0.86773, Lambda_2: 0.00449
Epoch: 1300 Loss: 2.83670e-03, Lambda_1: 0.88440, Lambda_2: 0.00444
Epoch: 1400 Loss: 2.50535e-03, Lambda_1: 0.87494, Lambda_2: 

In [37]:
lambda1 = float(pinn_model.lambda_1.data)
lambda2 = float(pinn_model.lambda_2.data)
print('Lambda1 Pred:', round(lambda1,8), '  ; Lambda1 Actual:', 1.0)
print('Lambda2 Pred:', round(lambda2,8), '  ; Lambda2 Actual:', round(0.01/np.pi,8))

Lambda1 Pred: 0.98847651   ; Lambda1 Actual: 1.0
Lambda2 Pred: 0.00328013   ; Lambda2 Actual: 0.0031831
